# 🎯 Classificação de Spam usando MLP (Perceptron Multicamadas)

## Objetivo
Treinar uma rede neural MLP para classificar mensagens como **Spam** ou **Não Spam (Ham)**

### Especificações:
- **Dataset**: SMS/Email Spam
- **Features**: TF-IDF (5000 palavras)
- **Validação**: 10-fold Cross-validation
- **Métricas**: Acurácia, Precisão, Recall, F1-Score, Matriz de Confusão


## 1️⃣ Importação de Bibliotecas


In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Pré-processamento de texto
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import LabelEncoder

# Modelo MLP
from sklearn.neural_network import MLPClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações visuais
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")


## 2️⃣ Carregamento e Exploração dos Dados


In [ ]:
# Carregar o dataset
df = pd.read_csv('archive/combined_data.csv')

print("📊 Informações do Dataset:")
print(f"Total de mensagens: {len(df)}")
print(f"\nPrimeiras linhas:")
df.head()


In [ ]:
# Verificar valores nulos
print("🔍 Valores nulos:")
print(df.isnull().sum())

# Remover valores nulos se existirem
df = df.dropna()

print(f"\n✅ Dataset limpo: {len(df)} mensagens")


In [ ]:
# Distribuição das classes
print("📊 Distribuição das Classes:")
print(df['label'].value_counts())
print(f"\nProporção:")
print(df['label'].value_counts(normalize=True) * 100)

# Visualizar distribuição
plt.figure(figsize=(8, 5))
df['label'].value_counts().plot(kind='bar', color=['#2ecc71', '#e74c3c'])
plt.title('Distribuição das Classes', fontsize=14, fontweight='bold')
plt.xlabel('Classe (0=Ham, 1=Spam)', fontsize=12)
plt.ylabel('Quantidade', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


## 3️⃣ Preparação dos Dados


In [ ]:
# Separar features (X) e target (y)
X = df['text']
y = df['label']

print(f"📝 Total de mensagens: {len(X)}")
print(f"🎯 Total de labels: {len(y)}")
print(f"\n✅ Dados preparados!")


## 4️⃣ Extração de Features com TF-IDF


In [ ]:
# Configurar TF-IDF Vectorizer
# max_features=5000: Limita ao vocabulário de 5000 palavras mais frequentes
# max_df=0.8: Ignora palavras que aparecem em mais de 80% dos documentos
# min_df=2: Ignora palavras que aparecem em menos de 2 documentos

print("⚙️ Configurando TF-IDF Vectorizer...")
tfidf = TfidfVectorizer(
    max_features=5000,
    max_df=0.8,
    min_df=2,
    stop_words='english',
    ngram_range=(1, 2)  # Unigramas e bigramas
)

print("🔄 Transformando textos em features TF-IDF...")
X_tfidf = tfidf.fit_transform(X)

print(f"\n✅ Features TF-IDF criadas!")
print(f"📊 Dimensões da matriz: {X_tfidf.shape}")
print(f"   - {X_tfidf.shape[0]} mensagens")
print(f"   - {X_tfidf.shape[1]} features (palavras/bigramas)")


## 5️⃣ Divisão dos Dados (Hold-out para teste final)


In [ ]:
# Dividir em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Mantém a proporção das classes
)

print("📊 Divisão dos Dados:")
print(f"   - Treino: {X_train.shape[0]} mensagens ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   - Teste:  {X_test.shape[0]} mensagens ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\n✅ Dados divididos!")


## 6️⃣ Criação e Treinamento do Modelo MLP


In [ ]:
# Configurar o MLP Classifier
print("🤖 Criando modelo MLP...\n")

mlp = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # 2 camadas ocultas: 100 e 50 neurônios
    activation='relu',             # Função de ativação ReLU
    solver='adam',                 # Otimizador Adam
    max_iter=200,                  # Máximo de iterações
    random_state=42,
    early_stopping=True,           # Para quando não houver melhora
    validation_fraction=0.1,       # 10% dos dados de treino para validação
    verbose=True                   # Mostrar progresso
)

print("🎯 Configurações do MLP:")
print(f"   - Camadas ocultas: {mlp.hidden_layer_sizes}")
print(f"   - Função de ativação: {mlp.activation}")
print(f"   - Otimizador: {mlp.solver}")
print(f"   - Early stopping: {mlp.early_stopping}")
print(f"\n⏳ Treinando o modelo...\n")

# Treinar o modelo
mlp.fit(X_train, y_train)

print(f"\n✅ Modelo treinado com sucesso!")
print(f"📈 Iterações realizadas: {mlp.n_iter_}")


## 7️⃣ Avaliação com 10-Fold Cross-Validation


In [ ]:
print("🔄 Realizando 10-Fold Cross-Validation...\n")
print("⏳ Isso pode demorar alguns minutos...\n")

# Realizar cross-validation com múltiplas métricas
scoring = ['accuracy', 'precision', 'recall', 'f1']

cv_results = cross_validate(
    mlp, 
    X_train, 
    y_train, 
    cv=10,
    scoring=scoring,
    return_train_score=True,
    verbose=1
)

print("\n✅ Cross-Validation completo!\n")
print("="*60)
print("📊 RESULTADOS DO 10-FOLD CROSS-VALIDATION")
print("="*60)

for metric in scoring:
    train_scores = cv_results[f'train_{metric}']
    test_scores = cv_results[f'test_{metric}']
    
    print(f"\n{metric.upper()}:")
    print(f"   Treino: {train_scores.mean():.4f} (± {train_scores.std():.4f})")
    print(f"   Validação: {test_scores.mean():.4f} (± {test_scores.std():.4f})")

print("\n" + "="*60)


In [ ]:
# Visualizar resultados do Cross-Validation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Resultados do 10-Fold Cross-Validation', fontsize=16, fontweight='bold')

metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_keys = ['accuracy', 'precision', 'recall', 'f1']

for idx, (ax, metric_name, metric_key) in enumerate(zip(axes.flat, metrics_names, metrics_keys)):
    train_scores = cv_results[f'train_{metric_key}']
    test_scores = cv_results[f'test_{metric_key}']
    
    folds = np.arange(1, 11)
    
    ax.plot(folds, train_scores, 'o-', label='Treino', linewidth=2, markersize=6)
    ax.plot(folds, test_scores, 's-', label='Validação', linewidth=2, markersize=6)
    
    ax.axhline(test_scores.mean(), color='red', linestyle='--', 
               label=f'Média Val: {test_scores.mean():.4f}', alpha=0.7)
    
    ax.set_xlabel('Fold', fontsize=11)
    ax.set_ylabel(metric_name, fontsize=11)
    ax.set_title(f'{metric_name} por Fold', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xticks(folds)

plt.tight_layout()
plt.show()


## 8️⃣ Avaliação no Conjunto de Teste


In [ ]:
# Fazer predições no conjunto de teste
print("🔮 Fazendo predições no conjunto de teste...\n")
y_pred = mlp.predict(X_test)

# Calcular todas as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("="*60)
print("🎯 MÉTRICAS NO CONJUNTO DE TESTE")
print("="*60)
print(f"\n✨ Acurácia:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"✨ Precisão:  {precision:.4f} ({precision*100:.2f}%)")
print(f"✨ Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"✨ F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
print("\n" + "="*60)


In [ ]:
# Relatório de classificação detalhado
print("\n📋 RELATÓRIO DE CLASSIFICAÇÃO DETALHADO\n")
print(classification_report(y_test, y_pred, target_names=['Ham (Não Spam)', 'Spam']))


## 9️⃣ Matriz de Confusão


In [ ]:
# Calcular matriz de confusão
cm = confusion_matrix(y_test, y_pred)

print("📊 Matriz de Confusão:")
print(cm)
print(f"\nInterpretação:")
print(f"   - Verdadeiros Negativos (Ham correto): {cm[0, 0]}")
print(f"   - Falsos Positivos (Ham previsto como Spam): {cm[0, 1]}")
print(f"   - Falsos Negativos (Spam previsto como Ham): {cm[1, 0]}")
print(f"   - Verdadeiros Positivos (Spam correto): {cm[1, 1]}")


In [ ]:
# Visualizar matriz de confusão
fig, ax = plt.subplots(figsize=(10, 8))

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=['Ham (Não Spam)', 'Spam']
)

disp.plot(cmap='Blues', ax=ax, values_format='d', colorbar=True)
plt.title('Matriz de Confusão - Classificação de Spam', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## 🔟 Visualização das Métricas Finais


In [ ]:
# Comparar métricas Cross-Validation vs Teste
metrics_comparison = {
    'Métrica': ['Acurácia', 'Precisão', 'Recall', 'F1-Score'],
    'Cross-Validation (média)': [
        cv_results['test_accuracy'].mean(),
        cv_results['test_precision'].mean(),
        cv_results['test_recall'].mean(),
        cv_results['test_f1'].mean()
    ],
    'Teste Final': [accuracy, precision, recall, f1]
}

df_comparison = pd.DataFrame(metrics_comparison)
print("\n📊 COMPARAÇÃO: Cross-Validation vs Teste Final\n")
print(df_comparison.to_string(index=False))


In [ ]:
# Gráfico de barras comparativo
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_comparison['Métrica']))
width = 0.35

bars1 = ax.bar(x - width/2, df_comparison['Cross-Validation (média)'], 
               width, label='Cross-Validation (10-fold)', alpha=0.8)
bars2 = ax.bar(x + width/2, df_comparison['Teste Final'], 
               width, label='Teste Final', alpha=0.8)

ax.set_xlabel('Métricas', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Comparação de Métricas: Cross-Validation vs Teste Final', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(df_comparison['Métrica'])
ax.legend(fontsize=11)
ax.set_ylim([0, 1.05])
ax.grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


## 1️⃣1️⃣ Testando o Modelo com Exemplos


In [ ]:
# Função para testar novas mensagens
def predict_spam(message):
    """
    Prediz se uma mensagem é spam ou não
    """
    # Transformar a mensagem usando o TF-IDF já treinado
    message_tfidf = tfidf.transform([message])
    
    # Fazer predição
    prediction = mlp.predict(message_tfidf)[0]
    probability = mlp.predict_proba(message_tfidf)[0]
    
    result = "SPAM 🚫" if prediction == 1 else "HAM (Não Spam) ✅"
    confidence = probability[prediction] * 100
    
    print(f"Mensagem: {message}")
    print(f"Predição: {result}")
    print(f"Confiança: {confidence:.2f}%")
    print(f"Probabilidades: Ham={probability[0]:.3f}, Spam={probability[1]:.3f}")
    print("-" * 80)

# Testar com exemplos
print("\n🧪 TESTANDO O MODELO COM EXEMPLOS\n")
print("=" * 80)

exemplos = [
    "Congratulations! You won a free iPhone! Click here to claim now!",
    "Hey, are we still meeting for lunch tomorrow?",
    "URGENT: Your account will be closed. Verify now to avoid suspension!",
    "Hi mom, I'll be home late tonight. Don't wait for dinner.",
    "Get rich quick! Make $5000 per day working from home!"
]

for exemplo in exemplos:
    predict_spam(exemplo)
    print()


## 1️⃣2️⃣ Resumo Final do Projeto


In [ ]:
print("="*80)
print("📝 RESUMO FINAL DO PROJETO")
print("="*80)
print(f"\n🎯 Objetivo: Classificação de Spam usando MLP")
print(f"\n📊 Dataset:")
print(f"   - Total de mensagens: {len(df):,}")
print(f"   - Mensagens Ham: {(y == 0).sum():,}")
print(f"   - Mensagens Spam: {(y == 1).sum():,}")
print(f"\n🔧 Pré-processamento:")
print(f"   - Extração de features: TF-IDF")
print(f"   - Vocabulário: 5000 palavras/bigramas")
print(f"   - Stop words: Removidas (inglês)")
print(f"\n🤖 Modelo MLP:")
print(f"   - Arquitetura: {mlp.hidden_layer_sizes}")
print(f"   - Função de ativação: {mlp.activation}")
print(f"   - Otimizador: {mlp.solver}")
print(f"   - Iterações: {mlp.n_iter_}")
print(f"\n📈 Validação:")
print(f"   - Método: 10-Fold Cross-Validation")
print(f"   - Acurácia média: {cv_results['test_accuracy'].mean():.4f} (± {cv_results['test_accuracy'].std():.4f})")
print(f"\n✨ Resultados no Teste Final:")
print(f"   - Acurácia:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   - Precisão:  {precision:.4f} ({precision*100:.2f}%)")
print(f"   - Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"   - F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
print(f"\n" + "="*80)
print("\n✅ Projeto concluído com sucesso!")
print("🎥 Pronto para o vídeo explicativo!")
